## 데이콘 추석 선물 수요량 예측 AI 경진대회

개요: 주어진 데이터로 추석 선물 수요량을 예측한다.  
**Dataset Info.**  
**train.csv**    
ID : 학습 샘플의 고유 ID  
수요량 : 학습 샘플의 target  


**test.csv**
ID : 추론 샘플의 고유 ID  

**sample_submission.csv**  - 제출 양식  
ID : 추론 샘플의 고유 ID
수요량 : 수요 샘플의 target


In [23]:
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [24]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [25]:
train_df

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337
...,...,...,...,...,...,...,...,...,...,...
5867,TRAIN_5867,2,쇼핑몰 46,225000,0,도시 16,지역 3,쇼핑몰 유형 1,고급한과선물세트,27
5868,TRAIN_5868,2,쇼핑몰 42,62000,0,도시 16,지역 4,쇼핑몰 유형 3,특선스페셜스팸선물세트,40
5869,TRAIN_5869,2,쇼핑몰 43,131000,0,도시 5,지역 4,쇼핑몰 유형 1,명품맛김선물세트,55
5870,TRAIN_5870,2,쇼핑몰 7,85000,0,도시 7,지역 1,쇼핑몰 유형 1,실속형견과류선물세트,231


In [26]:
test_df

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,TEST_0000,2,쇼핑몰 71,113000,0,도시 16,지역 3,쇼핑몰 유형 1,상주반건시곶감선물세트
1,TEST_0001,0,쇼핑몰 57,224000,0,도시 6,지역 1,쇼핑몰 유형 2,고급한과선물세트
2,TEST_0002,2,쇼핑몰 38,150000,0,도시 16,지역 1,쇼핑몰 유형 2,최고의선택스팸선물세트
3,TEST_0003,1,쇼핑몰 50,132000,0,도시 16,지역 3,쇼핑몰 유형 1,재래김특선세트
4,TEST_0004,0,쇼핑몰 55,146000,0,도시 5,지역 4,쇼핑몰 유형 2,명품샤인머스캣선물세트
...,...,...,...,...,...,...,...,...,...
3910,TEST_3910,1,쇼핑몰 2,220000,1,도시 2,지역 2,쇼핑몰 유형 3,최고급송이버섯선물세트
3911,TEST_3911,0,쇼핑몰 38,142000,0,도시 16,지역 1,쇼핑몰 유형 2,명품반건시곶감선물세트
3912,TEST_3912,2,쇼핑몰 2,225000,0,도시 2,지역 2,쇼핑몰 유형 3,프리미엄한과선물세트
3913,TEST_3913,1,쇼핑몰 67,221000,1,도시 6,지역 1,쇼핑몰 유형 1,유기농송이버섯선물세트


In [27]:
train_data = train_df.drop(['ID', '수요량'], axis =1)
train_label = train_df['수요량']

test_data = test_df.drop('ID', axis=1)

In [28]:
# 결측치 확인
train_data.isna().sum()

추석까지 남은 기간(주)    0
쇼핑몰 구분           0
가격(원)            0
프로모션 여부          0
도시 유형            0
지역 유형            0
쇼핑몰 유형           0
선물 유형            0
dtype: int64

In [29]:
train_label.isna().sum()

0

In [30]:
test_data.isna().sum()

추석까지 남은 기간(주)    0
쇼핑몰 구분           0
가격(원)            0
프로모션 여부          0
도시 유형            0
지역 유형            0
쇼핑몰 유형           0
선물 유형            0
dtype: int64

In [31]:
train_data.keys()

Index(['추석까지 남은 기간(주)', '쇼핑몰 구분', '가격(원)', '프로모션 여부', '도시 유형', '지역 유형',
       '쇼핑몰 유형', '선물 유형'],
      dtype='object')

In [32]:
# Label Encoding
nominal_feature = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']

for feature in nominal_feature:
    le = LabelEncoder()
    le = le.fit(train_data[feature])
    train_data[feature] = le.transform(train_data[feature])

    for label in np.unique(test_data[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_data[feature] = le.transform(test_data[feature])

In [33]:
train_data['가격(원)'].dtypes

dtype('int64')

In [34]:
train_data['도시 유형'].dtypes

dtype('int64')

In [35]:
train_data['추석까지 남은 기간(주)'].dtypes

dtype('int64')

In [36]:
features_to_scale = train_data[['가격(원)', '추석까지 남은 기간(주)']]


# 객체 생성 및 학습
scaler = MinMaxScaler()
scaler.fit(features_to_scale)

normalized_features = scaler.transform(features_to_scale)

train_data[['가격(원)', '추석까지 남은 기간(주)']] = normalized_features

In [37]:
train_data

,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,0.5,6,0.576159,0,20,0,1,9
1,1.0,69,0.248344,0,13,0,0,15
2,0.0,6,0.096026,0,20,0,1,22
3,0.5,4,0.556291,0,3,2,0,32
4,0.5,61,0.337748,0,7,1,1,30
...,...,...,...,...,...,...,...,...
5867,1.0,40,0.619205,0,7,2,0,5
5868,1.0,36,0.079470,0,7,3,2,40
5869,1.0,37,0.307947,0,19,3,0,11
5870,1.0,66,0.155629,0,21,0,0,23


In [38]:
# test_data scaling
features_to_scale_test_data = test_data[['가격(원)', '추석까지 남은 기간(주)']]

# 객체 생성 및 학습
scaler_test_data = MinMaxScaler()
scaler_test_data.fit(features_to_scale_test_data)

normalized_features_test_data = scaler.transform(features_to_scale_test_data)

test_data[['가격(원)', '추석까지 남은 기간(주)']] = normalized_features_test_data

In [39]:
"""
# 훈련 데이터와 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size=0.2, random_state=42)
"""

'\n# 훈련 데이터와 검증 데이터 분리\nX_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size=0.2, random_state=42)\n'

## 모델링  
GradientBoostingRegressor  
GridSearchCV로 최적의 하이퍼파라미터 찾기

In [40]:
# 그래디언트 부스팅 회귀 모델
model = GradientBoostingRegressor(random_state=42)

# 튜닝할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_data, train_label)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [41]:
# 최적의 하이퍼파라미터와 모델 출력
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(f'Best Hyperparameters: {best_params}')

Best Hyperparameters: {'max_depth': 5, 'n_estimators': 300}


In [43]:
result_df = pd.read_csv('sample_submission.csv')
result_df['수요량'] = best_model.predict(test_data)
result_df.head()

,ID,수요량
0,TEST_0000,200.592489
1,TEST_0001,36.507932
2,TEST_0002,294.043586
3,TEST_0003,214.349803
4,TEST_0004,278.100825


In [44]:
result_df.to_csv('result_submission7.csv', index=False)